# 通过时间的反向传播
:label:`sec_bptt`

如果你完成了 :numref:`sec_rnn-scratch` 中的练习，你会看到梯度裁剪对于防止偶尔出现的巨大梯度破坏训练是至关重要的。我们暗示过，爆炸的梯度源于长序列上的反向传播。在介绍一系列现代RNN架构之前，让我们更仔细地看看*反向传播*在序列模型中的数学细节。希望这一讨论能够为*消失*和*爆炸*梯度的概念带来一些精确性。如果你还记得我们在引入MLP时讨论过的计算图的前向和后向传播（:numref:`sec_backprop`），那么RNN中的前向传播应该相对简单。在RNN中应用反向传播被称为*通过时间的反向传播* :cite:`Werbos.1990`。这个过程要求我们将RNN的计算图按时间步展开（或展开）。展开的RNN本质上是一个前馈神经网络，其特殊之处在于相同的参数在整个展开网络中重复出现，在每个时间步上都出现。然后，就像在任何前馈神经网络中一样，我们可以应用链式法则，将梯度反向传播通过展开的网络。关于每个参数的梯度必须在其在展开网络中出现的所有位置求和。处理这种权重绑定应该从我们的卷积神经网络章节中熟悉。

复杂性来源于序列可以相当长。处理超过一千个标记的文本序列并不罕见。请注意，这从计算（内存过多）和优化（数值不稳定）的角度都带来了问题。来自第一步的输入需要经过超过1000次矩阵乘法才能到达输出，而计算梯度还需要另外1000次矩阵乘法。我们现在分析可能出错的地方以及如何在实践中解决它。


## RNN中的梯度分析
:label:`subsec_bptt_analysis`

我们从一个简化的RNN工作模型开始。该模型忽略了有关隐藏状态及其更新的具体细节。这里的数学符号没有明确区分标量、向量和矩阵。我们只是试图发展一些直觉。在这个简化模型中，我们用$h_t$表示隐藏状态，$x_t$表示输入，$o_t$表示时间步$t$的输出。回顾我们在:numref:`subsec_rnn_w_hidden_states`中的讨论，输入和隐藏状态可以在被隐藏层的一个权重变量相乘之前连接在一起。因此，我们使用$w_\textrm{h}$和$w_\textrm{o}$来分别表示隐藏层和输出层的权重。结果，每个时间步的隐藏状态和输出为

$$\begin{aligned}h_t &= f(x_t, h_{t-1}, w_\textrm{h}),\\o_t &= g(h_t, w_\textrm{o}),\end{aligned}$$
:eqlabel:`eq_bptt_ht_ot`

其中$f$和$g$分别是隐藏层和输出层的变换。因此，我们有一系列值$\{\ldots, (x_{t-1}, h_{t-1}, o_{t-1}), (x_{t}, h_{t}, o_t), \ldots\}$，它们通过递归计算相互依赖。前向传播相当直接。我们只需要一次一个时间步地遍历$(x_t, h_t, o_t)$三元组。目标函数跨所有$T$个时间步评估输出$o_t$与期望目标$y_t$之间的差异：

$$L(x_1, \ldots, x_T, y_1, \ldots, y_T, w_\textrm{h}, w_\textrm{o}) = \frac{1}{T}\sum_{t=1}^T l(y_t, o_t).$$



对于反向传播，事情有点棘手，特别是当我们计算目标函数$L$相对于参数$w_\textrm{h}$的梯度时。具体来说，根据链式法则，

$$\begin{aligned}\frac{\partial L}{\partial w_\textrm{h}}  & = \frac{1}{T}\sum_{t=1}^T \frac{\partial l(y_t, o_t)}{\partial w_\textrm{h}}  \\& = \frac{1}{T}\sum_{t=1}^T \frac{\partial l(y_t, o_t)}{\partial o_t} \frac{\partial g(h_t, w_\textrm{o})}{\partial h_t}  \frac{\partial h_t}{\partial w_\textrm{h}}.\end{aligned}$$
:eqlabel:`eq_bptt_partial_L_wh`

:eqref:`eq_bptt_partial_L_wh`中的乘积的第一和第二个因子很容易计算。第三个因子$\partial h_t/\partial w_\textrm{h}$则变得棘手，因为我们需要递归地计算参数$w_\textrm{h}$对$h_t$的影响。根据:eqref:`eq_bptt_ht_ot`中的递归计算，$h_t$既取决于$h_{t-1}$也取决于$w_\textrm{h}$，其中$h_{t-1}$的计算也依赖于$w_\textrm{h}$。因此，使用链式法则评估$h_t$相对于$w_\textrm{h}$的全导数得到

$$\frac{\partial h_t}{\partial w_\textrm{h}}= \frac{\partial f(x_{t},h_{t-1},w_\textrm{h})}{\partial w_\textrm{h}} +\frac{\partial f(x_{t},h_{t-1},w_\textrm{h})}{\partial h_{t-1}} \frac{\partial h_{t-1}}{\partial w_\textrm{h}}.$$
:eqlabel:`eq_bptt_partial_ht_wh_recur`


为了推导上述梯度，假设我们有三个序列$\{a_{t}\},\{b_{t}\},\{c_{t}\}$满足$a_{0}=0$且$a_{t}=b_{t}+c_{t}a_{t-1}$对于$t=1, 2,\ldots$。那么对于$t\geq 1$，容易证明

$$a_{t}=b_{t}+\sum_{i=1}^{t-1}\left(\prod_{j=i+1}^{t}c_{j}\right)b_{i}.$$
:eqlabel:`eq_bptt_at`

通过将$a_t$、$b_t$和$c_t$替换为

$$\begin{aligned}a_t &= \frac{\partial h_t}{\partial w_\textrm{h}},\\
b_t &= \frac{\partial f(x_{t},h_{t-1},w_\textrm{h})}{\partial w_\textrm{h}}, \\
c_t &= \frac{\partial f(x_{t},h_{t-1},w_\textrm{h})}{\partial h_{t-1}},\end{aligned}$$

:eqref:`eq_bptt_partial_ht_wh_recur`中的梯度计算满足$a_{t}=b_{t}+c_{t}a_{t-1}$。因此，根据:eqref:`eq_bptt_at`，我们可以去掉:eqref:`eq_bptt_partial_ht_wh_recur`中的递归计算，用

$$\frac{\partial h_t}{\partial w_\textrm{h}}=\frac{\partial f(x_{t},h_{t-1},w_\textrm{h})}{\partial w_\textrm{h}}+\sum_{i=1}^{t-1}\left(\prod_{j=i+1}^{t} \frac{\partial f(x_{j},h_{j-1},w_\textrm{h})}{\partial h_{j-1}} \right) \frac{\partial f(x_{i},h_{i-1},w_\textrm{h})}{\partial w_\textrm{h}}.$$
:eqlabel:`eq_bptt_partial_ht_wh_gen`

虽然我们可以使用链式法则递归地计算$\partial h_t/\partial w_\textrm{h}$，但当$t$很大时，这个链可以变得非常长。让我们讨论几种处理这个问题的策略。

### 完整计算 ### 

一种想法可能是计算:eqref:`eq_bptt_partial_ht_wh_gen`中的完整和。然而，这非常慢，并且梯度可能会爆炸，因为初始条件的细微变化可能会极大地影响结果。也就是说，我们可能会看到类似于蝴蝶效应的现象，即初始条件的微小变化会导致结果的不成比例的变化。这通常是不希望的。毕竟，我们正在寻找泛化良好的稳健估计器。因此，这种策略几乎从未在实践中使用。

### 截断时间步###

另一种方法是在:eqref:`eq_bptt_partial_ht_wh_gen`中截断和之后的$\tau$步。这是我们迄今为止一直在讨论的内容。这导致了真实梯度的一个*近似*，只需在$\partial h_{t-\tau}/\partial w_\textrm{h}$处终止和即可。实际上，这效果很好。这就是通常所说的截断的通过时间的反向传播 :cite:`Jaeger.2002`。这样做的一个后果是模型主要关注短期影响而不是长期后果。这实际上是*可取的*，因为它偏向于更简单和更稳定的模型。

### 随机截断 ### 

最后，我们可以用一个期望正确的随机变量代替$\partial h_t/\partial w_\textrm{h}$，但会截断序列。这是通过使用预先定义的$0 \leq \pi_t \leq 1$的一系列$\xi_t$实现的，其中$P(\xi_t = 0) = 1-\pi_t$和$P(\xi_t = \pi_t^{-1}) = \pi_t$，因此$E[\xi_t] = 1$。我们用它来替换:eqref:`eq_bptt_partial_ht_wh_recur`中的梯度$\partial h_t/\partial w_\textrm{h}$

$$z_t= \frac{\partial f(x_{t},h_{t-1},w_\textrm{h})}{\partial w_\textrm{h}} +\xi_t \frac{\partial f(x_{t},h_{t-1},w_\textrm{h})}{\partial h_{t-1}} \frac{\partial h_{t-1}}{\partial w_\textrm{h}}.$$


根据$\xi_t$的定义可知$E[z_t] = \partial h_t/\partial w_\textrm{h}$。每当$\xi_t = 0$时，递归计算在该时间步$t$终止。这导致不同长度序列的加权和，其中长序列很少见但适当加权。这个想法是由:citet:`Tallec.Ollivier.2017`提出的。

### 比较策略

![比较RNN中计算梯度的策略。从上到下：随机截断、常规截断和完整计算。](../img/truncated-bptt.svg)
:label:`fig_truncated_bptt`


:numref:`fig_truncated_bptt`说明了在使用通过时间的反向传播分析《时间机器》的前几个字符时的三种策略：

* 第一行是随机截断，将文本划分为不同长度的段。
* 第二行是常规截断，将文本分成相同长度的子序列。这是我们一直在RNN实验中做的事情。
* 第三行是完整的通过时间的反向传播，导致在计算上不可行的表达式。


不幸的是，尽管理论上很有吸引力，但随机截断的效果并不比常规截断好多少，可能是因为多种因素。首先，观察在多次反向传播后的过去的影响已经足够捕捉实际中的依赖关系。其次，增加的方差抵消了梯度随着更多步骤变得更准确的事实。第三，我们实际上*想要*只有短程交互的模型。因此，常规截断的通过时间的反向传播具有轻微的正则化效果，这可能是理想的。

## 通过时间的反向传播详细说明

在讨论了一般原则之后，让我们详细讨论通过时间的反向传播。与:numref:`subsec_bptt_analysis`中的分析相比，以下我们将展示如何计算目标函数相对于所有分解模型参数的梯度。为了保持简单，我们考虑一个没有偏置参数的RNN，其隐藏层的激活函数使用恒等映射（$\phi(x)=x$）。对于时间步$t$，令单个示例输入和目标分别为$\mathbf{x}_t \in \mathbb{R}^d$和$y_t$。隐藏状态$\mathbf{h}_t \in \mathbb{R}^h$和输出$\mathbf{o}_t \in \mathbb{R}^q$计算如下

$$\begin{aligned}\mathbf{h}_t &= \mathbf{W}_\textrm{hx} \mathbf{x}_t + \mathbf{W}_\textrm{hh} \mathbf{h}_{t-1},\\
\mathbf{o}_t &= \mathbf{W}_\textrm{qh} \mathbf{h}_{t},\end{aligned}$$

其中$\mathbf{W}_\textrm{hx} \in \mathbb{R}^{h \times d}$，$\mathbf{W}_\textrm{hh} \in \mathbb{R}^{h \times h}$，和$\mathbf{W}_\textrm{qh} \in \mathbb{R}^{q \times h}$是权重参数。记$l(\mathbf{o}_t, y_t)$为时间步$t$的损失。我们的目标函数，即从序列开始的$T$个时间步的损失为

$$L = \frac{1}{T} \sum_{t=1}^T l(\mathbf{o}_t, y_t).$$


为了可视化RNN在计算过程中模型变量和参数之间的依赖关系，我们可以绘制模型的计算图，如:numref:`fig_rnn_bptt`所示。例如，时间步3的隐藏状态$\mathbf{h}_3$的计算依赖于模型参数$\mathbf{W}_\textrm{hx}$和$\mathbf{W}_\textrm{hh}$、前一时间步的隐藏状态$\mathbf{h}_2$以及当前时间步的输入$\mathbf{x}_3$。

![显示具有三个时间步的RNN模型依赖关系的计算图。方框代表变量（未阴影）或参数（阴影），圆圈代表运算符。](../img/rnn-bptt.svg)
:label:`fig_rnn_bptt`

正如刚刚提到的，:numref:`fig_rnn_bptt`中的模型参数是$\mathbf{W}_\textrm{hx}$、$\mathbf{W}_\textrm{hh}$和$\mathbf{W}_\textrm{qh}$。一般来说，训练这个模型需要计算这些参数的梯度$\partial L/\partial \mathbf{W}_\textrm{hx}$、$\partial L/\partial \mathbf{W}_\textrm{hh}$和$\partial L/\partial \mathbf{W}_\textrm{qh}$。根据:numref:`fig_rnn_bptt`中的依赖关系，我们可以沿着箭头的相反方向依次计算并存储梯度。为了灵活表达链式法则中不同形状的矩阵、向量和标量的乘法，我们继续使用在:numref:`sec_backprop`中描述的$\textrm{prod}$运算符。

首先，相对于任何时间步$t$的模型输出对目标函数进行求导是非常直接的：

$$\frac{\partial L}{\partial \mathbf{o}_t} =  \frac{\partial l (\mathbf{o}_t, y_t)}{T \cdot \partial \mathbf{o}_t} \in \mathbb{R}^q.$$
:eqlabel:`eq_bptt_partial_L_ot`

现在我们可以计算相对于输出层参数$\mathbf{W}_\textrm{qh}$的目标梯度$\partial L/\partial \mathbf{W}_\textrm{qh} \in \mathbb{R}^{q \times h}$。基于:numref:`fig_rnn_bptt`，目标$L$通过$\mathbf{o}_1, \ldots, \mathbf{o}_T$依赖于$\mathbf{W}_\textrm{qh}$。使用链式法则得到

$$
\frac{\partial L}{\partial \mathbf{W}_\textrm{qh}}
= \sum_{t=1}^T \textrm{prod}\left(\frac{\partial L}{\partial \mathbf{o}_t}, \frac{\partial \mathbf{o}_t}{\partial \mathbf{W}_\textrm{qh}}\right)
= \sum_{t=1}^T \frac{\partial L}{\partial \mathbf{o}_t} \mathbf{h}_t^\top,
$$

其中$\partial L/\partial \mathbf{o}_t$由:eqref:`eq_bptt_partial_L_ot`给出。

接下来，如:numref:`fig_rnn_bptt`所示，在最终时间步$T$，目标函数$L$仅通过$\mathbf{o}_T$依赖于隐藏状态$\mathbf{h}_T$。因此，我们可以很容易地使用链式法则找到梯度$\partial L/\partial \mathbf{h}_T \in \mathbb{R}^h$：

$$\frac{\partial L}{\partial \mathbf{h}_T} = \textrm{prod}\left(\frac{\partial L}{\partial \mathbf{o}_T}, \frac{\partial \mathbf{o}_T}{\partial \mathbf{h}_T} \right) = \mathbf{W}_\textrm{qh}^\top \frac{\partial L}{\partial \mathbf{o}_T}.$$
:eqlabel:`eq_bptt_partial_L_hT_final_step`

对于任何时间步$t < T$，情况变得更加复杂，其中目标函数$L$通过$\mathbf{h}_{t+1}$和$\mathbf{o}_t$依赖于$\mathbf{h}_t$。根据链式法则，任何时间步$t < T$的隐藏状态梯度$\partial L/\partial \mathbf{h}_t \in \mathbb{R}^h$可以递归计算为：

$$\frac{\partial L}{\partial \mathbf{h}_t} = \textrm{prod}\left(\frac{\partial L}{\partial \mathbf{h}_{t+1}}, \frac{\partial \mathbf{h}_{t+1}}{\partial \mathbf{h}_t} \right) + \textrm{prod}\left(\frac{\partial L}{\partial \mathbf{o}_t}, \frac{\partial \mathbf{o}_t}{\partial \mathbf{h}_t} \right) = \mathbf{W}_\textrm{hh}^\top \frac{\partial L}{\partial \mathbf{h}_{t+1}} + \mathbf{W}_\textrm{qh}^\top \frac{\partial L}{\partial \mathbf{o}_t}.$$
:eqlabel:`eq_bptt_partial_L_ht_recur`

为了分析，对于任何时间步$1 \leq t \leq T$展开递归计算得到

$$\frac{\partial L}{\partial \mathbf{h}_t}= \sum_{i=t}^T {\left(\mathbf{W}_\textrm{hh}^\top\right)}^{T-i} \mathbf{W}_\textrm{qh}^\top \frac{\partial L}{\partial \mathbf{o}_{T+t-i}}.$$
:eqlabel:`eq_bptt_partial_L_ht`

从:eqref:`eq_bptt_partial_L_ht`可以看出，这个简单的线性例子已经表现出长序列模型的一些关键问题：它涉及$\mathbf{W}_\textrm{hh}^\top$的潜在非常大的幂。在其中，小于1的特征值消失，大于1的特征值发散。这是数值不稳定的，表现为消失和爆炸的梯度。一种解决方法是如:numref:`subsec_bptt_analysis`中讨论的那样，出于计算方便和数值稳定性的考虑截断时间步。在实践中，这种截断也可以通过在给定数量的时间步后分离梯度来实现。稍后，我们将看到更复杂的序列模型，如长短时记忆网络，如何进一步缓解这个问题。

最后，:numref:`fig_rnn_bptt`显示目标函数$L$通过隐藏状态$\mathbf{h}_1, \ldots, \mathbf{h}_T$依赖于隐藏层中的模型参数$\mathbf{W}_\textrm{hx}$和$\mathbf{W}_\textrm{hh}$。为了计算相对于这些参数的梯度$\partial L / \partial \mathbf{W}_\textrm{hx} \in \mathbb{R}^{h \times d}$和$\partial L / \partial \mathbf{W}_\textrm{hh} \in \mathbb{R}^{h \times h}$，我们应用链式法则得到

$$
\begin{aligned}
\frac{\partial L}{\partial \mathbf{W}_\textrm{hx}}
&= \sum_{t=1}^T \textrm{prod}\left(\frac{\partial L}{\partial \mathbf{h}_t}, \frac{\partial \mathbf{h}_t}{\partial \mathbf{W}_\textrm{hx}}\right)
= \sum_{t=1}^T \frac{\partial L}{\partial \mathbf{h}_t} \mathbf{x}_t^\top,\\
\frac{\partial L}{\partial \mathbf{W}_\textrm{hh}}
&= \sum_{t=1}^T \textrm{prod}\left(\frac{\partial L}{\partial \mathbf{h}_t}, \frac{\partial \mathbf{h}_t}{\partial \mathbf{W}_\textrm{hh}}\right)
= \sum_{t=1}^T \frac{\partial L}{\partial \mathbf{h}_t} \mathbf{h}_{t-1}^\top,
\end{aligned}
$$

其中$\partial L/\partial \mathbf{h}_t$由:eqref:`eq_bptt_partial_L_hT_final_step`和:eqref:`eq_bptt_partial_L_ht_recur`递归计算，是影响数值稳定性的关键量。

由于通过时间的反向传播是RNN中反向传播的应用，正如我们在:numref:`sec_backprop`中解释的那样，训练RNN交替进行前向传播和通过时间的反向传播。此外，通过时间的反向传播依次计算并存储上述梯度。具体来说，存储的中间值被重用来避免重复计算，例如存储$\partial L/\partial \mathbf{h}_t$用于计算$\partial L / \partial \mathbf{W}_\textrm{hx}$和$\partial L / \partial \mathbf{W}_\textrm{hh}$。


## 总结

通过时间的反向传播仅仅是带有隐藏状态的序列模型的反向传播的应用。出于计算便利性和数值稳定性，需要进行截断，如常规截断或随机截断。矩阵的高次幂可能导致特征值发散或消失。这表现为梯度爆炸或消失。为了高效计算，通过时间的反向传播期间缓存中间值。


## 练习

1. 假设我们有一个对称矩阵$\mathbf{M} \in \mathbb{R}^{n \times n}$，其特征值为$\lambda_i$，对应的特征向量为$\mathbf{v}_i$（$i = 1, \ldots, n$）。不失一般性，假设它们按照$|\lambda_i| \geq |\lambda_{i+1}|$的顺序排列。
   1. 证明$\mathbf{M}^k$的特征值为$\lambda_i^k$。
   1. 证明对于一个随机向量$\mathbf{x} \in \mathbb{R}^n$，以很高的概率$\mathbf{M}^k \mathbf{x}$将非常接近$\mathbf{M}$的特征向量$\mathbf{v}_1$。形式化这个陈述。
   1. 上述结果对RNN中的梯度意味着什么？
1. 除了梯度裁剪之外，你还能想到其他应对循环神经网络中梯度爆炸的方法吗？

[讨论](https://discuss.d2l.ai/t/334)